In [1]:
# LangGraph

from dotenv import dotenv_values
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph


config = dotenv_values(".env")

model = ChatOpenAI(api_key=config["OPEN_AI_KEY"], temperature=0)

In [4]:
# The graph below contains a single node called "oracle" that executes a chat model, then returns the result

graph = MessageGraph()

graph.add_node("oracle", model)
graph.add_edge("oracle", END)

graph.set_entry_point("oracle")

runnable = graph.compile()

In [5]:
# Run
runnable.invoke(HumanMessage("What is 1 + 1?"))

[HumanMessage(content='What is 1 + 1?', id='83e8e5f0-6f31-4651-b036-fe7419ab5a85'),
 AIMessage(content='1 + 1 equals 2.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-45fcc772-760b-4abe-a0d5-542871ce2c01-0')]

In [8]:
# Conditional edges
# Using tool calling

from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool
def multiply(first_number: int, second_number: int):
    """Multiplies two numbers together."""
    return first_number * second_number

model = ChatOpenAI(api_key=config["OPEN_AI_KEY"], temperature=0)
model_with_tools = model.bind_tools([multiply])

builder = MessageGraph()

builder.add_node("oracle", model_with_tools)

tool_node = ToolNode([multiply])
builder.add_node("multiply", tool_node)

builder.add_edge("multiply", END)

builder.set_entry_point("oracle")


In [14]:
# We can achieve this using conditional edges, which call a function on the current state and routes execution to a node the function's output

from typing import Literal
from typing import List
from langchain.schema import BaseMessage

def router(state: List[BaseMessage]) -> Literal["multiply", "__end__"]:
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "__end__"

builder.add_conditional_edges("oracle", router)

In [15]:
runnable = builder.compile()

runnable.invoke(HumanMessage("What is 123 * 456?"))

[HumanMessage(content='What is 123 * 456?', id='a1630dc9-afa4-4a46-b89d-39e18863650e'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_LcOiiG7i5gRJvYvGo3RSjQNL', 'function': {'arguments': '{"first_number":123,"second_number":456}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 69, 'total_tokens': 88}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-7284c62f-a4e6-4849-95d7-c94998978331-0', tool_calls=[{'name': 'multiply', 'args': {'first_number': 123, 'second_number': 456}, 'id': 'call_LcOiiG7i5gRJvYvGo3RSjQNL'}]),
 ToolMessage(content='56088', name='multiply', id='06dfea89-efc7-4717-b06d-4ff26ab2fc38', tool_call_id='call_LcOiiG7i5gRJvYvGo3RSjQNL')]

In [17]:
# If the model output contains a tool call, we move to the "multiply" node. Otherwise, we end execution
# Conversational responses are outputted directly

runnable.invoke(HumanMessage("What is your name?"))

[HumanMessage(content='What is your name?', id='1a000182-b88b-4b76-adfe-7da0e210a021'),
 AIMessage(content='My name is Assistant. How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 66, 'total_tokens': 79}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-cffdc2b5-37f1-4850-93cf-ef04953940c0-0')]

In [5]:
# Cycles

# # DELETE THIS
# %env TAVILY_API_KEY="tvly"

from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

env: TAVILY_API_KEY="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"


In [6]:
# Wrap these tools in a simple LangGraph ToolNode

from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools)

In [7]:
from langchain_openai import ChatOpenAI

# We will set streaming=True so that we can stream tokens
# See the streaming section
model = ChatOpenAI(api_key=config["OPEN_AI_KEY"], model="gpt-3.5-turbo", temperature=0, streaming=True)

In [8]:
# Make sure the model knows that it has these tools available to call

model = model.bind_tools(tools)

In [10]:
from typing import TypedDict, Annotated

def add_messages(left: list, right: list):
    """Add-don't-overwrite."""
    return left + right

class AgentState(TypedDict):
    # The `add_messages` function within the annotation defines
    # *how* updates should be merged into the state.
    messages: Annotated[list, add_messages]

In [11]:
# Define the nodes

###
# Conditional Edge: after the agent is called, we should either:

# a. Run tools if the agent said to take an action, OR

# b. Finish (respond to the user) if the agent did not ask to run tools

# Normal Edge: after the tools are invoked, the graph should always return to the agent to decide what to do next
###

from typing import Literal

# Define the function that determines whether to continue or not
def should_continue(state: AgentState) -> Literal["action", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]
    # If the LLM makes a tool call, then we route to the "action" node
    if last_message.tool_calls:
        return "action"
    # Otherwise, we stop (reply to the user)
    return "__end__"


# Define the function that calls the model
def call_model(state: AgentState):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}



In [12]:
# Define the Graph

from langgraph.graph import StateGraph, END
# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("action", tool_node)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.set_entry_point("agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
)

# We now add a normal edge from `tools` to `agent`.
# This means that after `tools` is called, `agent` node is called next.
workflow.add_edge('action', 'agent')

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

In [13]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in sf'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_w5yXd61MyyxSNeHkNPKAAw9A', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-3a27ea63-dc1f-4188-9b04-0efd96a6297d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_w5yXd61MyyxSNeHkNPKAAw9A'}]),
  ToolMessage(content="HTTPError('400 Client Error: Bad Request for url: https://api.tavily.com/search')", name='tavily_search_results_json', tool_call_id='call_w5yXd61MyyxSNeHkNPKAAw9A'),
  AIMessage(content='I encountered an error while trying to fetch the weather information for San Francisco. Let me try again.', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_KZvCK1zNa1vjPLhuHeErIJAO', 'function': {'arguments': '{"query":"weather in San F

In [ ]:
# Streaming
